# Лабораторная работа №1. Метод ближайших соседей (KNN)


## 1. Выбор данных

Для работы выбраны два датасета:

1. **Классификация**: `Mushrooms` (грибы).
   - Задача: определить класс гриба (съедобный/ядовитый).
   - Метрики: Accuracy, Confusion Matrix.

2. **Регрессия**: `Car Price Prediction` (автомобили).
   - Задача: предсказать стоимость автомобиля.
   - Метрики: MAE, RMSE, R2.


In [6]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    mean_absolute_error,
    mean_squared_error,
    r2_score
)


In [8]:
# Загрузка датасета грибов
mush = pd.read_csv("data/mushrooms.csv")

# Загрузка датасета автомобилей
cars = pd.read_csv("data/car_price.csv")

print("Mushrooms shape:", mush.shape)
print("Cars shape:", cars.shape)

display(mush.head())
display(cars.head())

Mushrooms shape: (8124, 23)
Cars shape: (1000, 8)


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


,Make,Model,Year,Engine Size,Mileage,Fuel Type,Transmission,Price
0,Honda,Model B,2015,3.9,74176,Petrol,Manual,30246.207931
1,Ford,Model C,2014,1.7,94799,Electric,Automatic,22785.747684
2,BMW,Model B,2006,4.1,98385,Electric,Manual,25760.290347
3,Honda,Model B,2015,2.6,88919,Electric,Automatic,25638.003491
4,Honda,Model C,2004,3.4,138482,Petrol,Automatic,21021.386657


In [12]:
print("=== Mushrooms: ===")
print("Строк:", mush.shape[0], "Столбцов:", mush.shape[1])
print("Типы данных:")
print(mush.dtypes.head())
print("Пропуски (первые 10 колонок):")
print(mush.isnull().sum().head(10))

print("\n=== Cars: ===")
print("Строк:", cars.shape[0], "Столбцов:", cars.shape[1])
print("Типы данных:")
print(cars.dtypes.head())
print("Пропуски:")
print(cars.isnull().sum())

=== Mushrooms: ===
Строк: 8124 Столбцов: 23
Типы данных:
class          object
cap-shape      object
cap-surface    object
cap-color      object
bruises        object
dtype: object
Пропуски (первые 10 колонок):
class              0
cap-shape          0
cap-surface        0
cap-color          0
bruises            0
odor               0
gill-attachment    0
gill-spacing       0
gill-size          0
gill-color         0
dtype: int64

=== Cars: ===
Строк: 1000 Столбцов: 8
Типы данных:
Make            object
Model           object
Year             int64
Engine Size    float64
Mileage          int64
dtype: object
Пропуски:
Make            0
Model           0
Year            0
Engine Size     0
Mileage         0
Fuel Type       0
Transmission    0
Price           0
dtype: int64


## 2. Первичный анализ данных

- **Mushrooms**: 8124 записи, все признаки категориальные, пропусков нет. Требуется One-Hot Encoding.
- **Cars**: 1000 записей, смешанные признаки (числовые + категориальные). Требуется масштабирование (StandardScaler) и кодирование (OHE).


## 3. Подготовка данных

Разбиваем на train/test и готовим пайплайны предобработки.


In [13]:
X_mush = mush.drop("class", axis=1)
y_mush = mush["class"]

cat_features_mush = X_mush.columns.tolist()

X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_mush,
    y_mush,
    test_size=0.2,
    random_state=42,
    stratify=y_mush
)

print("Mushrooms train/test:", X_train_m.shape, X_test_m.shape)


TARGET_COL = "Price"

X_cars = cars.drop(TARGET_COL, axis=1)
y_cars = cars[TARGET_COL]

num_features_cars = X_cars.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_features_cars = X_cars.select_dtypes(include=["object"]).columns.tolist()

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_cars,
    y_cars,
    test_size=0.2,
    random_state=42
)

print("Cars train/test:", X_train_c.shape, X_test_c.shape)


Mushrooms train/test: (6499, 22) (1625, 22)
Cars train/test: (800, 7) (200, 7)


In [14]:
# Препроцессор для грибов: OneHotEncoder для всех признаков
preprocess_mush = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features_mush)
    ]
)

# Препроцессор для автомобилей: StandardScaler + OneHotEncoder
preprocess_cars = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features_cars),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features_cars)
    ]
)


## 4. Бейзлайн (Sklearn)

Обучаем `KNeighborsClassifier` и `KNeighborsRegressor` с параметрами по умолчанию.


In [16]:
# Бейзлайн KNN для грибов
knn_clf_baseline = Pipeline(steps=[
    ("preprocess", preprocess_mush),
    ("model", KNeighborsClassifier())
])

knn_clf_baseline.fit(X_train_m, y_train_m)

y_pred_m = knn_clf_baseline.predict(X_test_m)

acc_m = accuracy_score(y_test_m, y_pred_m)
print("=== KNN Baseline (Mushrooms) ===")
print("Accuracy:", acc_m)

print("\nClassification report:")
print(classification_report(y_test_m, y_pred_m))

print("\nConfusion matrix:")
print(confusion_matrix(y_test_m, y_pred_m))


=== KNN Baseline (Mushrooms) ===
Accuracy: 1.0

Classification report:
              precision    recall  f1-score   support

           e       1.00      1.00      1.00       842
           p       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625


Confusion matrix:
[[842   0]
 [  0 783]]


In [18]:
# Бейзлайн KNN для автомобилей
knn_reg_baseline = Pipeline(steps=[
    ("preprocess", preprocess_cars),
    ("model", KNeighborsRegressor())
])

knn_reg_baseline.fit(X_train_c, y_train_c)

y_pred_c = knn_reg_baseline.predict(X_test_c)

mae_c = mean_absolute_error(y_test_c, y_pred_c)
rmse_c = mean_squared_error(y_test_c, y_pred_c) ** 0.5  # ручной RMSE
r2_c = r2_score(y_test_c, y_pred_c)

print("=== KNN Baseline (Cars) ===")
print("MAE:", mae_c)
print("RMSE:", rmse_c)
print("R^2:", r2_c)


=== KNN Baseline (Cars) ===
MAE: 2209.174511849922
RMSE: 2790.000403140126
R^2: 0.7155628381437322


## 4. Анализ результатов бейзлайна KNN

### 4.1. Классификация (Mushrooms)

Бейзлайн-модель KNN показала следующие результаты:

- **Accuracy: 1.0**
- F1-score для обоих классов: **1.00**
- Матрица ошибок без единой ошибки (идеальное разделение)

Такие показатели характерны для данного датасета: признаки в нём достаточно хорошо отделяют съедобные грибы от ядовитых, большинство алгоритмов действительно показывают на нём очень высокую точность.  
Другими словами, KNN в базовой конфигурации полностью справляется с задачей классификации.

### 4.2. Регрессия (Car Price Prediction)

Бейзлайн-модель KNN для задачи регрессии дала следующие метрики:

- **MAE ≈ 2209**
- **RMSE ≈ 2790**
- **R² ≈ 0.716**

Интерпретация:

- MAE около 2200 означает, что модель ошибается в цене автомобиля примерно на 2.2 тысячи условных единиц.
- RMSE немного выше, что ожидаемо: эта метрика сильнее штрафует большие отклонения.
- Значение R² ≈ 0.72 говорит о том, что модель объясняет примерно 72% вариации цены автомобиля.

Для простого KNN без подбора параметров результаты считаются удовлетворительными, но явно оставляют пространство для улучшения.  
В следующих шагах качество модели может быть повышено с помощью подбора гиперпараметров (`n_neighbors`, метрики расстояния, схемы взвешивания).

---

### Вывод по бейзлайну

- Для грибов KNN в базовых настройках решает задачу без ошибок, что соответствует известной структуре датасета.  
- Для регрессии по автомобилям качество разумное, но не оптимальное — улучшение возможно и ожидаемо.


In [19]:
# Пайплайн для поиска гиперпараметров на грибах
knn_clf_pipe = Pipeline(steps=[
    ("preprocess", preprocess_mush),
    ("model", KNeighborsClassifier())
])

# Сетка гиперпараметров
param_grid_clf = {
    "model__n_neighbors": [3, 5, 7, 9, 11],
    "model__weights": ["uniform", "distance"],
    "model__metric": ["euclidean", "manhattan"]
}

grid_clf = GridSearchCV(
    estimator=knn_clf_pipe,
    param_grid=param_grid_clf,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1
)

grid_clf.fit(X_train_m, y_train_m)

print("Best params (mushrooms):", grid_clf.best_params_)
best_clf = grid_clf.best_estimator_

y_pred_m_best = best_clf.predict(X_test_m)

print("\n=== Improved KNN (Mushrooms) ===")
print("Accuracy:", accuracy_score(y_test_m, y_pred_m_best))
print("\nClassification report:")
print(classification_report(y_test_m, y_pred_m_best))
print("\nConfusion matrix:")
print(confusion_matrix(y_test_m, y_pred_m_best))


e:\frame_mai\.venv\Lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [0.9995378  0.99984595 0.9995378  0.99969189 0.9993837  0.9995378
 0.99922966 0.9993837  0.99922966 0.9993837         nan 1.
        nan 0.99984595        nan 0.99984595        nan 0.9995378
        nan 0.9995378 ]
  warnings.warn(


Best params (mushrooms): {'model__metric': 'manhattan', 'model__n_neighbors': 3, 'model__weights': 'distance'}

=== Improved KNN (Mushrooms) ===
Accuracy: 1.0

Classification report:
              precision    recall  f1-score   support

           e       1.00      1.00      1.00       842
           p       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625


Confusion matrix:
[[842   0]
 [  0 783]]


In [20]:
# Пайплайн для поиска гиперпараметров на автомобилях
knn_reg_pipe = Pipeline(steps=[
    ("preprocess", preprocess_cars),
    ("model", KNeighborsRegressor())
])

# Сетка гиперпараметров
param_grid_reg = {
    "model__n_neighbors": [3, 5, 7, 11, 15],
    "model__weights": ["uniform", "distance"],
    "model__metric": ["euclidean", "manhattan"]
}

grid_reg = GridSearchCV(
    estimator=knn_reg_pipe,
    param_grid=param_grid_reg,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

grid_reg.fit(X_train_c, y_train_c)

print("Best params (cars):", grid_reg.best_params_)
best_reg = grid_reg.best_estimator_

y_pred_c_best = best_reg.predict(X_test_c)

mae_best = mean_absolute_error(y_test_c, y_pred_c_best)
rmse_best = mean_squared_error(y_test_c, y_pred_c_best) ** 0.5
r2_best = r2_score(y_test_c, y_pred_c_best)

print("\n=== Improved KNN (Cars) ===")
print("MAE:", mae_best)
print("RMSE:", rmse_best)
print("R^2:", r2_best)


Best params (cars): {'model__metric': 'euclidean', 'model__n_neighbors': 15, 'model__weights': 'distance'}

=== Improved KNN (Cars) ===
MAE: 2174.30472969056
RMSE: 2700.5243166174405
R^2: 0.7335142510546147


## 5. Улучшение бейзлайна (GridSearchCV)

Подберем `n_neighbors`, `weights` и метрику расстояния.

**Итоги:**
1.  **Грибы**: Качество осталось 1.0 (улучшать некуда).
2.  **Автомобили**:
    *   `n_neighbors` = 15, `metric` = euclidean, `weights` = distance.
    *   MAE снизился до ~2174.
    *   R2 вырос до ~0.73.


In [26]:
import numpy as np

class MyKNNBase:
    def __init__(self, n_neighbors=5):
        self.n_neighbors = n_neighbors
        self.X_train = None
        self.y_train = None
    
    def _to_dense(self, X):
        """
        Преобразуем входные данные в плотный numpy-массив.
        Если это разреженная матрица (scipy sparse), вызываем .toarray().
        """
        # sparse-матрицы имеют метод .toarray()
        if hasattr(X, "toarray"):
            X = X.toarray()
        return np.array(X)
    
    def fit(self, X, y):
        """
        Сохраняем обучающую выборку в виде плотного массива.
        """
        self.X_train = self._to_dense(X)
        self.y_train = np.array(y)
        return self
    
    def _compute_distances(self, X):
        """
        Вычисляем евклидовы расстояния между объектами X и обучающей выборкой.
        """
        X = self._to_dense(X)
        # X: (n_test, n_features), X_train: (n_train, n_features)
        dists = np.sqrt(((X[:, None, :] - self.X_train[None, :, :]) ** 2).sum(axis=2))
        return dists


class MyKNNClassifier(MyKNNBase):
    def predict(self, X):
        """
        Предсказание классов:
        - считаем расстояния
        - берем индексы k ближайших соседей
        - выбираем наиболее часто встречающийся класс
        """
        dists = self._compute_distances(X)
        neighbors_idx = np.argsort(dists, axis=1)[:, :self.n_neighbors]
        neighbors_labels = self.y_train[neighbors_idx]
        
        preds = []
        for labels in neighbors_labels:
            values, counts = np.unique(labels, return_counts=True)
            preds.append(values[np.argmax(counts)])
        return np.array(preds)


class MyKNNRegressor(MyKNNBase):
    def predict(self, X):
        """
        Предсказание числовых значений:
        - считаем расстояния
        - берем k ближайших соседей
        - возвращаем среднее по таргету
        """
        dists = self._compute_distances(X)
        neighbors_idx = np.argsort(dists, axis=1)[:, :self.n_neighbors]
        neighbors_targets = self.y_train[neighbors_idx]
        return neighbors_targets.mean(axis=1)


In [27]:
# Кодирование данных для грибов
X_train_m_enc = preprocess_mush.fit_transform(X_train_m)
X_test_m_enc = preprocess_mush.transform(X_test_m)

# Кодирование данных для автомобилей
X_train_c_enc = preprocess_cars.fit_transform(X_train_c)
X_test_c_enc = preprocess_cars.transform(X_test_c)

print("Encoded mushrooms shapes:", X_train_m_enc.shape, X_test_m_enc.shape)
print("Encoded cars shapes:", X_train_c_enc.shape, X_test_c_enc.shape)


Encoded mushrooms shapes: (6499, 117) (1625, 117)
Encoded cars shapes: (800, 18) (200, 18)


In [28]:
# Собственный KNN-классификатор для грибов
my_knn_clf = MyKNNClassifier(n_neighbors=5)
my_knn_clf.fit(X_train_m_enc, y_train_m)

y_pred_m_my = my_knn_clf.predict(X_test_m_enc)

print("=== MyKNN (Mushrooms) ===")
print("Accuracy:", accuracy_score(y_test_m, y_pred_m_my))
print("\nClassification report:")
print(classification_report(y_test_m, y_pred_m_my))
print("\nConfusion matrix:")
print(confusion_matrix(y_test_m, y_pred_m_my))


=== MyKNN (Mushrooms) ===
Accuracy: 1.0

Classification report:
              precision    recall  f1-score   support

           e       1.00      1.00      1.00       842
           p       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625


Confusion matrix:
[[842   0]
 [  0 783]]


In [29]:
# Собственный KNN-регрессор для автомобилей
my_knn_reg = MyKNNRegressor(n_neighbors=5)
my_knn_reg.fit(X_train_c_enc, y_train_c)

y_pred_c_my = my_knn_reg.predict(X_test_c_enc)

mae_my = mean_absolute_error(y_test_c, y_pred_c_my)
rmse_my = mean_squared_error(y_test_c, y_pred_c_my) ** 0.5  # старый sklearn, без squared=False
r2_my = r2_score(y_test_c, y_pred_c_my)

print("=== MyKNN (Cars) ===")
print("MAE:", mae_my)
print("RMSE:", rmse_my)
print("R^2:", r2_my)


=== MyKNN (Cars) ===
MAE: 2209.174511849922
RMSE: 2790.000403140126
R^2: 0.7155628381437322


## 6. Собственная реализация KNN

Реализованы классы `MyKNNClassifier` и `MyKNNRegressor`.
Результаты совпадают со sklearn (baseline), так как алгоритм детерминирован.


In [30]:
import numpy as np

class MyKNNBase:
    def __init__(self, n_neighbors=5, metric="euclidean"):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.X_train = None
        self.y_train = None
    
    def _to_dense(self, X):
        if hasattr(X, "toarray"):
            X = X.toarray()
        return np.array(X)
    
    def fit(self, X, y):
        self.X_train = self._to_dense(X)
        self.y_train = np.array(y)
        return self
    
    def _compute_distances(self, X):
        X = self._to_dense(X)
        if self.metric == "euclidean":
            dists = np.sqrt(((X[:, None, :] - self.X_train[None, :, :]) ** 2).sum(axis=2))
        elif self.metric == "manhattan":
            dists = np.abs(X[:, None, :] - self.X_train[None, :, :]).sum(axis=2)
        else:
            raise ValueError(f"Unknown metric: {self.metric}")
        return dists


class MyKNNClassifier(MyKNNBase):
    def predict(self, X):
        dists = self._compute_distances(X)
        neighbors_idx = np.argsort(dists, axis=1)[:, :self.n_neighbors]
        neighbors_labels = self.y_train[neighbors_idx]

        preds = []
        for labels in neighbors_labels:
            values, counts = np.unique(labels, return_counts=True)
            preds.append(values[np.argmax(counts)])
        return np.array(preds)


class MyKNNRegressor(MyKNNBase):
    def predict(self, X):
        dists = self._compute_distances(X)
        neighbors_idx = np.argsort(dists, axis=1)[:, :self.n_neighbors]
        neighbors_targets = self.y_train[neighbors_idx]
        return neighbors_targets.mean(axis=1)


In [32]:
# улучшенный MyKNNClassifier для грибов
my_knn_clf_best = MyKNNClassifier(
    n_neighbors=3,
    metric="manhattan"
)

my_knn_clf_best.fit(X_train_m_enc, y_train_m)
y_pred_m_my_best = my_knn_clf_best.predict(X_test_m_enc)

print("=== Improved MyKNN (Mushrooms) ===")
print("Accuracy:", accuracy_score(y_test_m, y_pred_m_my_best))
print(classification_report(y_test_m, y_pred_m_my_best))
print(confusion_matrix(y_test_m, y_pred_m_my_best))


=== Improved MyKNN (Mushrooms) ===
Accuracy: 1.0
              precision    recall  f1-score   support

           e       1.00      1.00      1.00       842
           p       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625

[[842   0]
 [  0 783]]


In [33]:
# улучшенный MyKNNRegressor для автомобилей
my_knn_reg_best = MyKNNRegressor(
    n_neighbors=15,
    metric="euclidean"
)

my_knn_reg_best.fit(X_train_c_enc, y_train_c)
y_pred_c_my_best = my_knn_reg_best.predict(X_test_c_enc)

mae_my_best = mean_absolute_error(y_test_c, y_pred_c_my_best)
rmse_my_best = mean_squared_error(y_test_c, y_pred_c_my_best)**0.5
r2_my_best = r2_score(y_test_c, y_pred_c_my_best)

print("=== Improved MyKNN (Cars) ===")
print("MAE:", mae_my_best)
print("RMSE:", rmse_my_best)
print("R^2:", r2_my_best)


=== Improved MyKNN (Cars) ===
MAE: 2217.487351157528
RMSE: 2734.5361357338043
R^2: 0.7267594580618701


## 7. Выводы

Сравнение моделей (Регрессия):

| Модель | MAE | R2 |
|---|---|---|
| Sklearn Baseline | 2209 | 0.716 |
| Sklearn Improved | 2174 | 0.734 |
| MyKNN | 2217 | 0.727 |

1.  Все модели показали идеальный результат на задаче классификации (датасет Mushrooms).
2.  На задаче регрессии подбор гиперпараметров (особенно взвешивание по расстоянию) улучшил результат.
3.  Собственная реализация работает корректно и показывает результаты, сопоставимые с библиотечной.
